## Ensemble: Logistic Regression & Neural Network (MLPClassifier)

Combining the Neural Network (MLP) and Logistic Regression models using an Ensemble technique. Specifically, a Soft Voting Classifier is highly effective here. It averages the predicted probabilities from both models (e.g., if the Neural Network says "90% Spam" and Logistic Regression says "70% Spam", the ensemble sees "80% Spam").

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Setup (Assuming your dataframe 'df' is loaded)
# Make sure to run your data loading and cleaning steps first!
df = pd.read_csv('dataset.csv')

if not df.empty:
    # Fill missing values instead of dropping
    df['subject'] = df['subject'].fillna('')
    df['body'] = df['body'].fillna('')
    df['text_combined'] = df['subject'] + " " + df['body']

    # --- STEP 2: EXTRACT FORENSIC FEATURES ---
    print("Extracting forensic features...")

    # Feature A: Capitalization Ratio (The "Shout" Factor)
    def get_caps_ratio(text):
        if len(text) == 0: return 0.0
        return sum(1 for c in text if c.isupper()) / len(text)

    df['caps_ratio'] = df['subject'].apply(get_caps_ratio)

    # Feature B: URL Count (Phishing relies on links)
    df['num_urls'] = df['body'].apply(lambda x: len(re.findall(r'http[s]?://', str(x))))

    # Feature C: Risk Word Density
    # Simple count of high-risk forensic triggers
    risk_words = ['urgent', 'verify', 'account', 'suspended', 'click', 'winner', 'security']
    def count_risk_words(text):
        return sum(1 for w in risk_words if w in str(text).lower())

    df['risk_score'] = df['text_combined'].apply(count_risk_words)

    # Clean text for the TF-IDF part
    df['clean_text'] = df['text_combined'].apply(lambda x: re.sub(r'[^a-z\s]', '', str(x).lower()))

    # Prepare Training Data
    X = df[['clean_text', 'caps_ratio', 'num_urls', 'risk_score']]
    y = df['label'].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# 3. THE WORD EMBEDDING PIPELINE
# Instead of just TF-IDF, we add TruncatedSVD.
# This compresses the sparse words into 100 "Dense Semantic Features"
# effectively creating a document embedding similar to averaging Word2Vec vectors.
text_embedding_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('embedding', TruncatedSVD(n_components=100, random_state=42))
])

# 4. Hybrid Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('text_vectors', text_embedding_pipeline, 'clean_text'),
        ('forensics', MinMaxScaler(), ['caps_ratio', 'num_urls', 'risk_score'])
    ]
)

# 5. Ensemble Model
ensemble = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000)),
        ('mlp', MLPClassifier(hidden_layer_sizes=(100,), max_iter=500))
    ],
    voting='soft'
)

# 6. Full Pipeline
model = Pipeline([
    ('prep', preprocessor),
    ('clf', ensemble)
])

# Train & Evaluate
model.fit(X_train, y_train)

# Predict Probabilities
y_prob = model.predict_proba(X_test)[:, 1]

# Apply Safe Threshold (0.90)
threshold = 0.90
y_pred_safe = (y_prob >= threshold).astype(int)

# Calculate Metrics
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_safe).ravel()
acc = accuracy_score(y_test, y_pred_safe)
fpr = fp / (fp + tn)

print(f"\n--- HYBRID MODEL RESULTS (Threshold {threshold}) ---")
print(f"Accuracy: {acc:.4f} (Goal: High)")
print(f"False Positive Rate: {fpr:.4f} (Goal: <0.01)")
print(f"False Positives: {fp}")
print(f"True Positives (Caught): {tp}")

if acc > 0.96 and fpr < 0.01:
    print("\nSUCCESS: You have achieved high accuracy AND low false positives!")
else:
    print("\nNote: Performance improved, but consider adding more data or features (like BERT) to push further.")

Extracting forensic features...

--- HYBRID MODEL RESULTS (Threshold 0.9) ---
Accuracy: 0.9216 (Goal: High)
False Positive Rate: 0.0038 (Goal: <0.01)
False Positives: 38
True Positives (Caught): 9144

Note: Performance improved, but consider adding more data or features (like BERT) to push further.


## Hybrid Forensic Model: TF-IDF --> Linguistic Forensic clues

Instead of just looking at words (TF-IDF), we will feed the model specific forensic clues we analyzed earlier:

1. Caps Ratio: (Spammers SHOUT).
2. URL Count: (Phishing has more links).
3. Risk Words: (Specific triggers like "verify", "urgent").

In [6]:
import pandas as pd
import numpy as np
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# --- 1. DATA LOADING ---
# We try to load your dataset first. If not found, we create synthetic data for demo.
try:
    # Try loading common filenames
    try:
        df = pd.read_csv('dataset.csv')
    except FileNotFoundError:
        print("dataset.csv not found, trying another fileset...")

    # Basic cleanup
    df.dropna(subset=['subject', 'body', 'label'], inplace=True)
    print(" Successfully loaded dataset.")

except:
    print("Dataset not found. Generating SYNTHETIC data for demonstration...")
    # Synthetic Data Generation (Fallback)
    subjects = ["Urgent: Account Suspended", "Weekly Meeting", "YOU WON $1000", "Project Update", "Verify Identity", "Lunch?"] * 200
    bodies = [
        "Click here to restore access.",
        "See attached minutes.",
        "Claim your prize now! Urgent!",
        "Deadline moved to Friday.",
        "Unauthorized login attempt detected.",
        "Want to grab tacos?"
    ] * 200
    labels = [1, 0, 1, 0, 1, 0] * 200
    df = pd.DataFrame({'subject': subjects, 'body': bodies, 'label': labels})

# Combine text for analysis
# Fill NaNs just in case
df['subject'] = df['subject'].fillna('')
df['body'] = df['body'].fillna('')
df['text_combined'] = df['subject'] + " " + df['body']

# --- 2. FORENSIC FEATURE EXTRACTION ---
print("Extracting forensic features...")

# Feature A: Capitalization Ratio (The "Shout" Factor)
def get_caps_ratio(text):
    s_text = str(text)
    if len(s_text) == 0: return 0.0
    return sum(1 for c in s_text if c.isupper()) / len(s_text)

df['caps_ratio'] = df['subject'].apply(get_caps_ratio)

# Feature B: URL Count
# Note: Phishing emails typically have more links than normal emails
df['num_urls'] = df['body'].apply(lambda x: len(re.findall(r'http[s]?://', str(x))))

# Feature C: Risk Word Score
# A simple "bag of threats" approach
risk_words = ['urgent', 'verify', 'account', 'suspended', 'click', 'winner', 'security', 'claim', 'immediate', 'access']
def get_risk_score(text):
    text_lower = str(text).lower()
    return sum(1 for w in risk_words if w in text_lower)

df['risk_score'] = df['text_combined'].apply(get_risk_score)

# Text Cleaning for LSA
def clean_text(text):
    return re.sub(r'[^a-z\s]', '', str(text).lower()).strip()

df['clean_text'] = df['text_combined'].apply(clean_text)

# --- 3. MODEL PIPELINE SETUP ---

# Part A: Text Pipeline (LSA)
# TF-IDF -> TruncatedSVD = Latent Semantic Analysis (LSA)
# This creates "Dense Vectors" similar to Word2Vec
text_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('svd', TruncatedSVD(n_components=100, random_state=42)) # 100 dimensions
])

# Part B: Column Transformer
# Processes text with LSA and numbers with MinMax scaling simultaneously
preprocessor = ColumnTransformer(
    transformers=[
        ('text_lsa', text_pipeline, 'clean_text'),
        ('forensics', MinMaxScaler(), ['caps_ratio', 'num_urls', 'risk_score'])
    ]
)

# Part C: The Ensemble Classifier
# Combining Logistic Regression (Stability) and MLP (Complex Patterns)
ensemble_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42)),
        ('mlp', MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42))
    ],
    voting='soft' # Soft voting averages the probabilities
)

# Part D: Final Pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', ensemble_clf)
])

# --- 4. TRAINING & EVALUATION ---
X = df[['clean_text', 'caps_ratio', 'num_urls', 'risk_score']]
y = df['label'].astype(int)

# Split 75/25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"Training LSA Hybrid Model on {len(X_train)} emails...")
model.fit(X_train, y_train)

# --- 5. HIGH-CONFIDENCE THRESHOLDING ---
# We use a 0.90 threshold to minimize False Positives
print("\n--- Evaluation Results (High-Confidence Threshold > 0.90) ---")

# Get probabilities (Class 1 = Spam)
y_prob = model.predict_proba(X_test)[:, 1]
threshold = 0.90
y_pred_safe = (y_prob >= threshold).astype(int)

# Metrics
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_safe).ravel()
acc = accuracy_score(y_test, y_pred_safe)
fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0

print(f"Accuracy:              {acc:.4f}")
print(f"False Positive Rate:   {fpr:.4f} (Goal: < 0.01)")
print(f"False Positives:       {fp} (Legitimate emails lost)")
print(f"True Positives:        {tp} (Spam caught)")

if fpr == 0:
    print("\n[SUCCESS] Zero False Positives achieved.")
elif fpr < 0.01:
    print("\n[SUCCESS] Extremely low False Positive Rate achieved.")
else:
    print("\n[ADVICE] FPR is still > 1%. Consider raising threshold to 0.95 or adding more forensic features.")

 Successfully loaded dataset.
Extracting forensic features...
Training LSA Hybrid Model on 61603 emails...

--- Evaluation Results (High-Confidence Threshold > 0.90) ---
Accuracy:              0.9210
False Positive Rate:   0.0039 (Goal: < 0.01)
False Positives:       39 (Legitimate emails lost)
True Positives:        9069 (Spam caught)

[SUCCESS] Extremely low False Positive Rate achieved.


## Enhance Ensemble Model: Logistic Regression + Neural Network (MLP) + Gradient Boosting

We will add a Gradient Boosting model to the ensemble. Unlike Logistic Regression, Gradient Boosting builds trees sequentially to fix previous errors, often finding subtle patterns that linear models miss.

In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Load Data
try:
    df = pd.read_csv('dataset.csv')
    df.dropna(subset=['subject', 'body', 'label'], inplace=True)
except:
    print("Error: Dataset not found.")
    df = pd.DataFrame()

if not df.empty:
    df['subject'] = df['subject'].fillna('')
    df['body'] = df['body'].fillna('')
    df['text_combined'] = df['subject'] + " " + df['body']

    # --- FORENSIC FEATURE EXTRACTION ---
    print("Extracting features...")

    # Caps Ratio
    def get_caps_ratio(text):
        if len(str(text)) == 0: return 0.0
        return sum(1 for c in str(text) if c.isupper()) / len(str(text))
    df['caps_ratio'] = df['subject'].apply(get_caps_ratio)

    # URL Count
    df['num_urls'] = df['body'].apply(lambda x: len(re.findall(r'http[s]?://', str(x))))

    # Risk Word Score
    risk_words = ['urgent', 'verify', 'account', 'suspended', 'click', 'winner', 'security', 'claim', 'immediate']
    df['risk_score'] = df['text_combined'].apply(lambda x: sum(1 for w in risk_words if w in str(x).lower()))

    # Clean Text
    df['clean_text'] = df['text_combined'].apply(lambda x: re.sub(r'[^a-z\s]', '', str(x).lower()))

    # --- ENHANCED PIPELINE ---

    # 1. Improved LSA: Increase dimensions to 300 to keep more semantic meaning
    text_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
        ('svd', TruncatedSVD(n_components=300, random_state=42))
    ])

    # 2. Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('text_lsa', text_pipeline, 'clean_text'),
            ('forensics', MinMaxScaler(), ['caps_ratio', 'num_urls', 'risk_score'])
        ]
    )

    # 3. Stronger Ensemble: Adding Gradient Boosting
    # Gradient Boosting is excellent at capturing non-linear patterns in mixed data
    ensemble = VotingClassifier(
        estimators=[
            ('lr', LogisticRegression(max_iter=1000, random_state=42)),
            ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
        ],
        voting='soft'
    )

    model = Pipeline([
        ('prep', preprocessor),
        ('clf', ensemble)
    ])

    # Split Data
    X = df[['clean_text', 'caps_ratio', 'num_urls', 'risk_score']]
    y = df['label'].astype(int)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    # Train
    print("Training Enhanced Hybrid Model (LR + MLP + Gradient Boosting)...")
    model.fit(X_train, y_train)

    # Evaluate
    y_prob = model.predict_proba(X_test)[:, 1]

    # Optimizing Threshold
    # We slightly lower the threshold to 0.85 to recover accuracy,
    # relying on the stronger model to keep FPR low.
    threshold = 0.85
    y_pred_safe = (y_prob >= threshold).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_safe).ravel()
    acc = accuracy_score(y_test, y_pred_safe)
    fpr = fp / (fp + tn)

    print(f"\n--- ENHANCED MODEL RESULTS (Threshold {threshold}) ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"False Positive Rate: {fpr:.4f}")
    print(f"False Positives: {fp}")
    print(f"True Positives: {tp}")

    if acc > 0.95 and fpr < 0.01:
        print("\n[SUCCESS] High Accuracy and Low FPR achieved.")

Extracting features...
Training Enhanced Hybrid Model (LR + MLP + Gradient Boosting)...

--- ENHANCED MODEL RESULTS (Threshold 0.85) ---
Accuracy: 0.9518
False Positive Rate: 0.0041
False Positives: 41
True Positives: 9704

[SUCCESS] High Accuracy and Low FPR achieved.


## Another approach: Long-Short Term Memory (LSTM)

**The Advantage**: Unlike TF-IDF (which ignores word order) or simple embeddings (which average meanings), LSTMs read the email sequentially. They understand the difference between "not urgent" and "urgent" or "bank account" and "river bank" based on the sequence. This makes them better at detecting context-based phishing that uses natural-sounding language to trick filters.

**The Trade-off**: LSTMs are computationally heavier and slower to train than the Ensemble model you just built. For many simple spam filters, an Ensemble of Logistic Regression + LightGBM is "good enough" and much faster.

In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

# 1. Load & Clean Data
df = pd.read_csv('dataset.csv')
df.dropna(subset=['body', 'label'], inplace=True)

# Combine subject/body if needed
df['text_combined'] = df['subject'] + " " + df['body']

def clean_text(text):
    return re.sub(r'[^a-z\s]', '', str(text).lower()).strip()

df['clean_text'] = df['text_combined'].apply(clean_text)

X = df['clean_text'].values
y = df['label'].values

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 2. Tokenization & Padding
# LSTMs need fixed-length sequences of numbers, not raw text
MAX_NB_WORDS = 5000      # Vocabulary size
MAX_SEQUENCE_LENGTH = 100 # Cut off emails after 100 words
EMBEDDING_DIM = 100      # Vector size for each word

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)

# 3. Build LSTM Model
print("Building LSTM Model...")
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM)) # Turn words into dense vectors
model.add(SpatialDropout1D(0.2))                  # Regularization
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # The LSTM Layer
model.add(Dense(1, activation='sigmoid'))         # Output layer (0-1 prob)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. Train
# Epochs=5 is usually enough for spam
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_split=0.1, verbose=1)

# 5. Evaluate with High Threshold (Low False Positive)
y_prob_lstm = model.predict(X_test_pad)

threshold = 0.90
y_pred_safe = (y_prob_lstm >= threshold).astype(int)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_safe).ravel()
acc = accuracy_score(y_test, y_pred_safe)
fpr = fp / (fp + tn)

print(f"\n--- LSTM RESULTS (Threshold {threshold}) ---")
print(f"Accuracy: {acc:.4f}")
print(f"False Positive Rate: {fpr:.4f}")

Building LSTM Model...
Epoch 1/5
870/870 [==============================] - 317s 359ms/step - loss: 0.1289 - accuracy: 0.9534 - val_loss: 0.0823 - val_accuracy: 0.9719
Epoch 2/5
870/870 [==============================] - 273s 313ms/step - loss: 0.0621 - accuracy: 0.9795 - val_loss: 0.0646 - val_accuracy: 0.9758
Epoch 3/5
870/870 [==============================] - 248s 286ms/step - loss: 0.0438 - accuracy: 0.9862 - val_loss: 0.0614 - val_accuracy: 0.9762
Epoch 4/5
870/870 [==============================] - 257s 295ms/step - loss: 0.0365 - accuracy: 0.9874 - val_loss: 0.0668 - val_accuracy: 0.9787
Epoch 5/5
645/645 [==============================] - 18s 26ms/step

--- LSTM RESULTS (Threshold 0.9) ---
Accuracy: 0.9748
False Positive Rate: 0.0152


In [11]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_safe).ravel()
acc = accuracy_score(y_test, y_pred_safe)
fpr = fp / (fp + tn)

print(f"\n--- ENHANCED MODEL RESULTS (Threshold {threshold}) ---")
print(f"Accuracy: {acc:.4f}")
print(f"False Positive Rate: {fpr:.4f}")
print(f"False Positives: {fp}")
print(f"True Positives: {tp}")



--- ENHANCED MODEL RESULTS (Threshold 0.9) ---
Accuracy: 0.9748
False Positive Rate: 0.0152
False Positives: 150
True Positives: 10375
